In [6]:
import sys
import re
import os
from operator import add
from pyspark.sql import SparkSession
from pyspark import SparkContext

def GetFileName(file_dir, suffix):
    fileNames =[]
    for root, dirs, files in os.walk(file_dir):
        for file in files:
            if os.path.splitext(file)[1] == suffix:
                fileNames.append(os.path.join(root, file))
    for i in range(0, len(fileNames)):
        fileNames[i] = fileNames[i].replace('\\', '/')
    return fileNames

def GetEntireFile(filenames):
    filenames = GetFileName('./../dataset/name', '.name')
    outPath = './../dataset/name/entire.tmp'
    if not os.path.exists(outPath):
        print('Entire file not exists.Now creating...')
        fout = open(outPath, 'a', encoding='utf-8')
        for filename in filenames:
            fin = open(filename, 'r', encoding='utf-8')
            doc = fin.read()
            fout.write(doc)
        fout.close()
    return outPath
    
def combine(L):
    output = set()
    for i  in range(0, len(L)):
        for j in range(0, len(L)):
            if L[i] != L[j]:
                output.add((L[i], L[j]))
    return list(output)

def drive(taskName):
    spark = SparkSession\
        .builder\
        .appName(taskName)\
        .getOrCreate()
    filenames = GetFileName('./../dataset/name', '.name')
    outPath = './../result/task2.out'
    if os.path.exists(outPath):
        os.remove(outPath)
        print("Remove exising file: " + outPath)
    inPath = GetEntireFile(filenames)
    #inPath = './../dataset/name/金庸01飞狐外传.name'
    #inPath = './../dataset/name/entire.name'
    mapreduce(inPath, spark)
    print("Done.")
    
def mapreduce(inPath,spark):
    print('Processing: ' + inPath)
    lines  = spark.read.text(inPath) \
            .rdd.map(lambda x:x[0]) \
            .map(lambda x: x.split('|'))
    #print(lines.collect())
    counts = lines.flatMap(combine).map(lambda x:(x,1)).reduceByKey(add)
    output = counts.sortByKey(ascending=True).collect()
    fout = open('./../result/task2.out', 'w', encoding='utf-8')
    for (word, count) in output:
        fout.write(word[0] + ',' + word[1] + ',' + str(count) + '\n')
    fout.close()
    
drive('Task2')

Remove exising file: ./../result/task2.out
Processing: ./../dataset/name/entire.tmp
Done.
